# Import the relevant Libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder
import foursquare_credentials as credentials
from sklearn.cluster import KMeans
# map rendering library
import folium 
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

print("Libraries successfully imported")

Libraries successfully imported


## Insert Wiki URL

In [2]:
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(wiki_url).text

## Beautiful Soup to Parse the url page

In [3]:
soup = BeautifulSoup(response, 'xml')

In [4]:
# Target table
table=soup.find('table')

## Create dataframe with three columns: PostalCode, Borough, and Neighborhood

In [5]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [6]:
df

,Postalcode,Borough,Neighbourhood


## Extracting information from the table and input into dataframe

In [7]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [8]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Ignore cells with a borough that is Not assigned.

In [9]:
df=df[df['Borough']!='Not assigned']

## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [10]:
df[df['Neighbourhood']=='Not assigned']=df['Borough']

/Users/gohjieda/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/gohjieda/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2444: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer(indexer, value)
/Users/gohjieda/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2424: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

## To solve the issue that more than one neighborhood can exist in one postal code area. 

In [11]:
# Group multiple neighbourhoods into one Postcode
temp=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp=temp.reset_index(drop=False)
temp.rename(columns={'Neighbourhood':'joined'},inplace=True)
# Join the new data frame
df_merge = pd.merge(df, temp, on='Postalcode')
# Drop the Neighbourhood column
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)
# Drop duplicates
df_merge.drop_duplicates(inplace=True)
# Rename columns
df_merge.rename(columns={'joined':'Neighbourhood'},inplace=True)

In [12]:
df_merge.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,Queen's Park,Queen's Park,Queen's Park


## Dataframe Shape

In [13]:
df_merge.shape

(103, 3)

In [14]:
df_merge

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,Queen's Park,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


## Import Geocoder

In [15]:
def get_geocode(postal_code):
    # Initialize variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

## Use Geospatial Data url provided by coursera 

In [16]:
geomap = pd.read_csv('http://cocl.us/Geospatial_data')

In [17]:
#Rename columns
geomap.rename(columns={'Postal Code':'Postalcode'},inplace=True)
# Merge both dataframes
geo_merged = pd.merge(geomap, df_merge, on='Postalcode')

In [18]:
geo_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [19]:
# Change sequence of data
geo_data=geo_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]

In [20]:
geo_data

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Explore and cluster the neighborhoods in Toronto. 
#### Work with only boroughs that contain the word Toronto and then replicate the same analysis to the New York City data. 

In [21]:
toronto = geo_data[geo_data['Borough'].str.contains("Toronto")]

In [22]:
toronto.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


## Foursquare Credentials

In [23]:
CLIENT_ID = credentials.CLIENT_ID # your Foursquare ID
CLIENT_SECRET = credentials.CLIENT_SECRET # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [24]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
toronto.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [27]:
toronto_nearby_ven = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [28]:
toronto_nearby_ven.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


## Number of Unique Categories

In [29]:
print('There are {} uniques categories.'.format(len(toronto_nearby_ven['Venue Category'].unique())))

There are 236 uniques categories.


In [30]:
toronto_nearby_ven.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,83,83,83,83,83,83
"Chinatown, Grange Park, Kensington Market",93,93,93,93,93,93
Christie,17,17,17,17,17,17


## Perform one hot encoding

In [31]:
toronto_onehot = pd.get_dummies(toronto_nearby_ven[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True)
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_nearby_ven['Neighborhood'].tolist() )
toronto_onehot.shape

(1700, 236)

### Group rows by neighborhoods and the mean of the frequency of occurrence of each category

In [32]:
toronto_group = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_group.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,...,0.0,0.020000,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.017857,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.00,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [33]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Display the top 15 venues for each neighborhood.

In [34]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for index in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(index+1, indicators[index]))
    except:
        columns.append('{}th Most Common Venue'.format(index+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_group['Neighborhood']

for index in np.arange(toronto_group.shape[0]):
    neighborhoods_venues_sorted.iloc[index, 1:] = most_common_venues(toronto_group.iloc[index, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Asian Restaurant,Burger Joint,Restaurant,Hotel,American Restaurant,Steakhouse,Bakery,Thai Restaurant,Sushi Restaurant,Breakfast Spot,Concert Hall,Pizza Place
1,Berczy Park,Coffee Shop,Beer Bar,Café,Steakhouse,Bakery,Italian Restaurant,Cocktail Bar,Cheese Shop,Seafood Restaurant,Farmers Market,Indian Restaurant,Bistro,Fish Market,Breakfast Spot,Jazz Club
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Gym,Furniture / Home Store,Convenience Store,Burrito Place,Sandwich Place,Stadium,Caribbean Restaurant,Italian Restaurant,Bar,Restaurant,Intersection,Bakery
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Smoke Shop,Spa,Fast Food Restaurant,Park,Garden Center
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane,Boutique,Boat or Ferry,Sculpture Garden,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


## Cluster Neighborhoods

In [35]:
# set number of clusters
kclusters = 5

toronto_group_cluster = toronto_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_group_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

### Create a new dataframe that includes the cluster as well as the top 15 venues for each neighborhood.

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Health Food Store,Pub,Other Great Outdoors,Trail,...,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,...,Cosmetics Shop,Brewery,Bubble Tea Shop,Café,Caribbean Restaurant,Restaurant,Pub,Diner,Dessert Shop,Spa
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3,Sandwich Place,Park,Ice Cream Shop,Burger Joint,...,Liquor Store,Board Shop,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Sushi Restaurant,Italian Restaurant,Food & Drink Shop,Movie Theater,Brewery
43,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,Italian Restaurant,American Restaurant,...,Clothing Store,Diner,Cheese Shop,Comfort Food Restaurant,Middle Eastern Restaurant,Bank,Sandwich Place,Seafood Restaurant,Brewery,Coworking Space
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Lake,Swim School,Bus Line,...,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [37]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,3,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Asian Restaurant,Burger Joint,Restaurant,Hotel,American Restaurant,Steakhouse,Bakery,Thai Restaurant,Sushi Restaurant,Breakfast Spot,Concert Hall,Pizza Place
1,3,Berczy Park,Coffee Shop,Beer Bar,Café,Steakhouse,Bakery,Italian Restaurant,Cocktail Bar,Cheese Shop,Seafood Restaurant,Farmers Market,Indian Restaurant,Bistro,Fish Market,Breakfast Spot,Jazz Club
2,3,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Gym,Furniture / Home Store,Convenience Store,Burrito Place,Sandwich Place,Stadium,Caribbean Restaurant,Italian Restaurant,Bar,Restaurant,Intersection,Bakery
3,3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Smoke Shop,Spa,Fast Food Restaurant,Park,Garden Center
4,3,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane,Boutique,Boat or Ferry,Sculpture Garden,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


## Visualize the resulting clusters

In [38]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [41]:
toronto_merged.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Health Food Store,Pub,Other Great Outdoors,Trail,...,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,...,Cosmetics Shop,Brewery,Bubble Tea Shop,Café,Caribbean Restaurant,Restaurant,Pub,Diner,Dessert Shop,Spa
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3,Sandwich Place,Park,Ice Cream Shop,Burger Joint,...,Liquor Store,Board Shop,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Sushi Restaurant,Italian Restaurant,Food & Drink Shop,Movie Theater,Brewery
43,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,Italian Restaurant,American Restaurant,...,Clothing Store,Diner,Cheese Shop,Comfort Food Restaurant,Middle Eastern Restaurant,Bank,Sandwich Place,Seafood Restaurant,Brewery,Coworking Space
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Lake,Swim School,Bus Line,...,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters